In [1]:
# install openai
!pip install -q openai

In [2]:
# importing enviornment dependencies
import os, re, json

In [3]:
#now we will be using groq api
#from google.colab import userdata

#Fetch Groq key from Colab secrets
Groq_Api_Key = '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'
if Groq_Api_Key is None:
    raise ValueError("Groq API key not found")

#Set environment variable for OpenAI/Groq client
os.environ["GROQ_API_KEY"] = Groq_Api_Key

In [4]:
from openai import OpenAI
Envio = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    )


In [5]:
#Task 1:
#Conversation Management
history = []   # just a list to keep chats
runs = 0       # count how many times
def add_msg(role, text):
  history.append({"role": role, "content": text})

In [6]:
#for truncation of history
def truncate_history(limit_type="turns", value=5):
  global history
  if limit_type == "turns":
    history = history[-value:]
  elif limit_type == "chars":
    # kinda messy but works
    total = 0
    newhist = []
    for msg in reversed(history):
      if total + len(msg["content"]) <= value:
        newhist.insert(0, msg)
        total += len(msg["content"])
    history = newhist

In [7]:
def summarize_history():
  # join all messages into text
  text = ""
  for m in history:
    text += m["role"] + ": " + m["content"] + "\n"

  res = Envio.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
      {"role": "system", "content": "make a short summary of this conversation"},
      {"role": "user", "content": text}
    ],
    temperature=0.3
  )
  return res.choices[0].message.content

In [8]:
def periodic_summary(k=3):
  global runs, history
  runs += 1
  if runs % k == 0:
    s = summarize_history()
    history = [{"role": "system", "content": "summary: " + s}]

In [9]:
#task 2: json schema extraction

schema = {
  "name": "extract_user_info",
  "description": "pull out name, email, phone, location, age",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {"type": "string"},
      "email": {"type": "string"},
      "phone": {"type": "string"},
      "location": {"type": "string"},
      "age": {"type": "string"}
    },
    "required": ["name", "email", "phone", "location", "age"]
  }
}

In [10]:
def extract_info(text):
  res = Envio.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": text}],
    tools=[{"type": "function", "function": schema}],
    tool_choice="auto"
  )
  tool = res.choices[0].message.tool_calls[0]
  return tool.function.arguments

In [11]:
# demo
add_msg("user", "Hello I am Karan, from Chandigarh.")
add_msg("assistant", "Hi Karan!")
add_msg("user", "my email is karan@mail.com and phone 1234566753. I am 90 years old.")
periodic_summary(3)  # after 3rd run, it should summarize
print("conversation history now:")
print(history)

conversation history now:
[{'role': 'user', 'content': 'Hello I am Karan, from Chandigarh.'}, {'role': 'assistant', 'content': 'Hi Karan!'}, {'role': 'user', 'content': 'my email is karan@mail.com and phone 1234566753. I am 90 years old.'}]


In [12]:
sample = "My name is Karan, I am 99, from Wakanda, email karan@gmail.com phone 999888777."
print("\nextracted info:")
print(extract_info(sample))


extracted info:
{"age":"99","email":"karan@gmail.com","location":"Wakanda","name":"Karan","phone":"999888777"}
